In [1]:
import pyopencl as cl

ImportError: DLL load failed: The specified procedure could not be found.

In [3]:
def GrabNVIDIA():
    NVIDIA = cl.get_platforms()[ 1 ]
    context = cl.Context( 
                     properties = [ ( cl.context_properties.PLATFORM, NVIDIA ) ], 
                     dev_type = cl.device_type.ALL )
    return context


In [4]:
gtx960M = GrabNVIDIA()

NameError: name 'cl' is not defined

In [5]:
gtx960M

NameError: name 'gtx960M' is not defined

In [ ]:
ctx = gtx960M

In [ ]:
import numpy as np

#Slightly modified from https://documen.tician.de/pyopencl/

a_np = np.random.rand(50000).astype(np.float32)
b_np = np.random.rand(50000).astype(np.float32)

queue = cl.CommandQueue(ctx)

mf = cl.mem_flags
a_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a_np)
b_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b_np)

prg = cl.Program(ctx, """
__kernel void sum(
    __global const float *a_g, __global const float *b_g, __global float *res_g)
{
  int gid = get_global_id(0);
  res_g[gid] = a_g[gid] + b_g[gid];
}
""").build()

res_g = cl.Buffer(ctx, mf.WRITE_ONLY, a_np.nbytes)
prg.sum(queue, a_np.shape, None, a_g, b_g, res_g)

res_np = np.empty_like(a_np)
cl.enqueue_copy(queue, res_np, res_g)

# Check on CPU with Numpy:
print(res_np - (a_np + b_np))
print(np.linalg.norm(res_np - (a_np + b_np)))